# imports

In [1]:
from pathlib import Path
import numpy as np
import xarray as xr
import h5py
from tqdm import tqdm
import math
from typing import Tuple
import os

import json
import ast


import sys
sys.path.append("C:/Users/GAI/Desktop/Scott/NCA_Research")


from core_utils.plotting import *
from E4_PI_NCA.utils.helper import *


from core_utils.utils_image import *

In [2]:

def print_h5_structure(obj, indent=0):
    """
    遞迴列印 h5 內容
    - dataset: print shape
    - group: 遞迴
    - attrs: print key/values
    - list/dict/tuple: print len()
    """
    prefix = "  " * indent
    if isinstance(obj, h5py.File) or isinstance(obj, h5py.Group):
        print(f"{prefix}{obj.name} (Group)")
        # 列印 attributes
        for k, v in obj.attrs.items():
            if isinstance(v, (list, tuple)):
                print(f"{prefix}  attr {k}: {type(v).__name__}, len={len(v)}")
            else:
                print(f"{prefix}  attr {k}: {type(v).__name__} -> {v}")
        # 遞迴列印子物件
        for key in obj:
            print_h5_structure(obj[key], indent + 1)
    elif isinstance(obj, h5py.Dataset):
        print(f"{prefix}{obj.name} (Dataset) shape={obj.shape} dtype={obj.dtype}")
    else:
        print(f"{prefix}{obj.name}: Unknown type {type(obj)}")



# Helper Functions

"""
將 dataset/ 下的所有 {case}/ 資料夾轉換成分城市的 HDF5 結構。

每個 city 對應一個 .h5 檔案，結構如下：

GlobalMetaData.h5

Cities/

    city_X.h5

        ├─ city_base (C,H,W)

        └─ cases/

            ├─ case_0/wind_field (C,H,W)

            ├─ case_0/meta (屬性)
"""

In [3]:
def remove_all_files(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path):
                os.remove(file_path)
            elif os.path.isdir(file_path):
                # 若要同時刪除子資料夾內的內容
                remove_all_files(file_path)
                os.rmdir(file_path)
        except Exception as e:
            print(f"Error deleting {file_path}: {e}")

In [4]:
# ---------------------- Helper Functions ----------------------
#region Helper Functions
def nan_to_masked_CHW(arr_CHW: np.ndarray):
    nan_mask = np.isnan(arr_CHW).any(axis=0)
    geo_mask = (~nan_mask).astype(np.float16)
    geo_mask = geo_mask[::-1]
    arr_clean = np.array(arr_CHW, copy=True)
    arr_clean[np.isnan(arr_clean)] = 0.0
    return arr_clean, geo_mask


def add_coord_channels(arr_CHW: np.ndarray, geo_mask: np.ndarray):
    C, H, W = arr_CHW.shape
    ys = np.linspace(-1.0, 1.0, H, dtype=np.float16)[:, None]
    xs = np.linspace(-1.0, 1.0, W, dtype=np.float16)[None, :]
    coord_y = np.repeat(ys, W, axis=1)
    coord_x = np.repeat(xs, H, axis=0)
    stacked = np.concatenate([
        coord_y[np.newaxis, ...],
        coord_x[np.newaxis, ...],
        geo_mask[np.newaxis, ...],
        arr_CHW
    ], axis=0)
    return stacked



def extract_city_name_and_wind(case_name: str) -> Tuple[str, float, float]:
    """
    從 case_name 提取 city 名稱與風向資訊
    風向角度以北為 0 度，逆時針，單位 degree
    回傳：
        city_name: str
        wind_dir_deg: float
        wind_xy: Tuple[float, float] (x, y)
    範例：
        case_name = "CN-BE-V1_d150"
        -> city_name="CN-BE-V1", wind_dir_deg=150, wind_xy=(sin, cos)
    """
    parts = case_name.split("_")
    city_name = parts[0] if len(parts) > 1 else "city_default"

    # 嘗試提取 _dXX
    wind_dir_deg = 0.0
    for p in parts:
        if p.startswith("d"):
            try:
                wind_dir_deg = float(p[1:])
            except:
                wind_dir_deg = 0.0
            break

    # 轉換成 xy 分量 (x:東, y:北)
    # 風向角以北為 0，逆時針
    wind_rad = math.radians(wind_dir_deg)
    wind_x = math.sin(wind_rad)
    wind_y = math.cos(wind_rad)

    return city_name, wind_dir_deg, (wind_x, wind_y)


def process_ped_file(ped_file: Path):
    """
    處理 ped.nc，回傳：
    - city_base_channels: 固定通道 (coord_y, coord_x, geo_mask, topo)
    - case_channels: 剩下的通道 (windInitX, windInitY, ped data)
    - city_base_channel_names
    - case_channel_names
    """
    case_name = ped_file.parent.name
    # 嘗試解析風向
    try:
        wind_dir = float(case_name.split("_d")[-1])
    except Exception:
        wind_dir = 0.0

    # 讀取 ped.nc -> HWC
    with xr.open_dataset(ped_file) as ds:
        arrays = [ds[var].values for var in ds.data_vars]
        ped_np = np.stack(arrays, axis=-1)  # (H,W,C)
        vars_names = list(ds.data_vars.keys())

    H, W, _ = ped_np.shape

    # 初始風場 (H,W,2)
    wind_np = np.zeros((H, W, 2), dtype=np.float32)
    wind_np[..., 0] = np.sin(np.deg2rad(wind_dir))
    wind_np[..., 1] = np.cos(np.deg2rad(wind_dir))

    # topo
    topo_file = next(ped_file.parent.glob("*_topo"), None)
    if topo_file:
        topo = np.loadtxt(topo_file)
        if topo.ndim == 2:
            topo = topo[:, :, np.newaxis]
        ped_np = np.concatenate([topo, wind_np, ped_np], axis=-1)
        channel_names = ["topo", "windInitX", "windInitY"] + vars_names
    else:
        ped_np = np.concatenate([wind_np, ped_np], axis=-1)
        channel_names = ["windInitX", "windInitY"] + vars_names

    # nan -> mask
    ped_np, geo_mask = nan_to_masked_CHW(ped_np.transpose(2,0,1))  # CHW
    # 加入 coord channels -> CHW
    ped_np = add_coord_channels(ped_np, geo_mask)

    # 新增 channel names
    channel_names = ["coord_y", "coord_x", "geo_mask"] + channel_names

    # 拆分 city_base 與 case_channels
    city_base_idx = ["coord_y", "coord_x", "geo_mask", "topo"]
    city_base_channels = []
    case_channels = []
    city_base_channel_names = []
    case_channel_names = []

    for i, name in enumerate(channel_names):
        if name in city_base_idx:
            city_base_channels.append(ped_np[i])
            city_base_channel_names.append(name)
        else:
            case_channels.append(ped_np[i])
            case_channel_names.append(name)

    city_base = np.stack(city_base_channels, axis=0)  # CHW
    case_data = np.stack(case_channels, axis=0)       # CHW

    return city_base.astype(np.float16), case_data.astype(np.float16), city_base_channel_names, case_channel_names,(H,W)




# main

In [5]:

# ==== config ====
input_folder = Path("../dataset")
root_out = Path("../dataset_h5")
train_dir = root_out / "train"
val_dir = root_out / "val"
train_ratio = 0.8

# ==== clean output ====
if root_out.exists():
    for p in root_out.rglob("*"):
        if p.is_file():
            p.unlink()
else:
    root_out.mkdir()

train_dir.mkdir(exist_ok=True, parents=True)
val_dir.mkdir(exist_ok=True, parents=True)

# === gather city cases ===
city_cases = {}
for ped_file in input_folder.rglob("*ped.nc"):
    case_name = ped_file.parent.name
    city_name, _, _ = extract_city_name_and_wind(case_name)
    city_cases.setdefault(city_name, []).append(ped_file)

print(f"📦 cities = {len(city_cases)}")

# === city-level split ===
all_cities = list(city_cases.keys())
random.shuffle(all_cities)

train_count = int(len(all_cities) * train_ratio)
train_cities = set(all_cities[:train_count])
val_cities = set(all_cities[train_count:])

print(f"train cities: {len(train_cities)} | val cities: {len(val_cities)}")

def save_city_to_h5(city_name, ped_files, out_path):
    with h5py.File(out_path, "w") as h5f:
        # 第一個case → city_base
        city_base_np, _, city_base_channels, _, _ = process_ped_file(ped_files[0])
        h5f.create_dataset("city_base", data=city_base_np.astype(np.float16), compression="gzip")

        cases_grp = h5f.create_group("cases")

        for ped_file in tqdm(ped_files, desc=f"{city_name}"):
            case_name = ped_file.parent.name
            _, case_data, _, case_ch, (H,W) = process_ped_file(ped_file)

            # flip Y if needed
            case_data = case_data[:, ::-1]

            _, wind_deg, (wx, wy) = extract_city_name_and_wind(case_name)
            meta = dict(
                cityHW=[H,W],
                wind_dir_deg=wind_deg,
                wind_dir_xy=[wx, wy],
                channels=case_ch
            )

            grp = cases_grp.create_group(case_name)
            grp.create_dataset("wind_field", data=case_data.astype(np.float16), compression="gzip")
            grp.attrs["meta"] = json.dumps(meta)

# === write h5 ===
for city_name, ped_files in city_cases.items():
    out_dir = train_dir if city_name in train_cities else val_dir
    out_path = out_dir / f"{city_name}.h5"
    save_city_to_h5(city_name, ped_files, out_path)

# === save metadata ===
def write_global_metadata(path, city_list):
    with h5py.File(path, "w") as f:
        f.attrs["num_cities"] = len(city_list)
        f.create_dataset("city_list",
            data=np.array([c.encode() for c in city_list], dtype="S"),
            compression="gzip")
        f.attrs["description"] = "GNCA wind-field dataset split by city"

write_global_metadata(train_dir / "GlobalMeta.h5", list(train_cities))
write_global_metadata(val_dir / "GlobalMeta.h5", list(val_cities))

print("✅ 完成 train/val 城市切分與 H5 建立")


📦 cities = 32
train cities: 25 | val cities: 7


NL-DEL-V2: 100%|██████████| 2/2 [00:00<00:00,  4.23it/s]

✅ 完成 train/val 城市切分與 H5 建立
